In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8')
sns.set(font_scale = 2.5)

import missingno as msno

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## 순서

1. 데이터셋 확인

2. EDA

3. feature engineering

4. model 만들기

5. 모델 학습 및 예측

6. 모델 평가

In [ ]:
path = '/content/drive/MyDrive/Kaggle/Kaggle_Competition/Code_Practice/dataset/titanic/'

In [ ]:
# 1. 데이터셋 확인

df_train = pd.read_csv(path + "train.csv")
df_test = pd.read_csv(path + "test.csv")

In [ ]:
df_train.head()

In [ ]:
df_train.describe()
df_test.describe()

In [ ]:
# 1.1 Null data check
for col in df_train.columns:
    msg = 'column: {:>10}\t Percent of NaN value {:.2f}%'.format(
        col , 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)

In [ ]:
for col in df_test.columns:
    msg = 'column: {:>10}\t Percent of NaN value {:.2f}%'.format(
        col , 100 * (df_test[col].isnull().sum() / df_test[col].shape[0]))
    print(msg)

In [ ]:
msno.matrix(df = df_train.iloc[: , :] ,
            figsize = (8 , 8) , color = (0.8 , 0.5 , 0.2))

In [ ]:
msno.bar(df = df_train.iloc[: , :] ,
         figsize = (8 , 8) , color = (0.8 , 0.5 , 0.2))

In [ ]:
msno.bar(df = df_test.iloc[: , :] ,
         figsize = (8 , 8) , color = (0.8 , 0.5 , 0.2))

In [ ]:
# 1.2 Target label 확인

f , ax = plt.subplots(1 , 2 , figsize = (18 , 8))

df_train['Survived'].value_counts().plot.pie(
    explode = [0 , 0.1] ,
    autopct = '%1.1f%%' ,
    ax = ax[0] ,
    shadow = True
)
ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('')
sns.countplot(x = df_train['Survived'] , ax = ax[1])
ax[1].set_title('Count plot - Survived')

plt.show()

In [ ]:
# 2. EDA

# 2.1 Pclass

df_train[['Pclass' , 'Survived']].groupby(['Pclass'] , as_index = True).count()

In [ ]:
df_train[['Pclass' , 'Survived']].groupby(['Pclass'] , as_index = True).sum()

In [ ]:
pd.crosstab(df_train['Pclass'] , df_train['Survived'] , margins = True).style.background_gradient(
    cmap = 'summer_r'
)

In [ ]:
df_train[['Pclass' , 'Survived']].groupby(['Pclass'] , as_index = True).mean().sort_values(by = 'Survived' , ascending = False).plot.bar()

In [ ]:
y_position = 1.02
f , ax = plt.subplots(1,  2 , figsize = (18 , 8))
df_train['Pclass'].value_counts().plot.bar(
    color = ['#CD7F32' , '#FFDF00' , '#D3D3D3'] , ax = ax[0]
)
ax[0].set_title('Number of Passengers By Pclass' , y = y_position)
ax[0].set_ylabel('Count')
sns.countplot(x = 'Pclass' , hue = 'Survived' , data = df_train , ax = ax[1])
ax[1].set_title('Pclass : Survived vs Dead' , y = y_position)

plt.show()

In [ ]:
# 2.2 Sex
f , ax = plt.subplots(1 , 2 , figsize = (18 , 8))
df_train[['Sex' , 'Survived']].groupby(['Sex'] , as_index = True).mean().plot.bar(ax = ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot(x = 'Sex' , hue = 'Survived' , data = df_train , ax = ax[1])
ax[1].set_title('Sex : Survived vs Dead')
plt.show()

In [ ]:
df_train[['Sex' , 'Survived']].groupby(['Sex'] , as_index = False).mean().sort_values(by = 'Survived' , ascending = False)

In [ ]:
pd.crosstab(df_train['Sex'] , df_train['Survived'] , margins = True).style.background_gradient(cmap = 'summer_r')

In [ ]:
# 2.3 Both Sex and Pclass
sns.pointplot(
    data = df_train ,
    x = 'Pclass' ,
    y = 'Survived' ,
    hue = 'Sex'
)
plt.legend(loc = 'upper right')

In [ ]:
# 2.4 Age
print('제일 나이 많은 탑승객 : {:.1f} Years'.format(df_train['Age'].max()))
print('제일 어린 탑승객 : {:.1f} Years'.format(df_train['Age'].min()))
print('탑승객 평균 나이 : {:.1f} Years'.format(df_train['Age'].mean()))

In [ ]:
fig , ax = plt.subplots(1 , 1 , figsize = (9 , 5))
sns.kdeplot(df_train[df_train['Survived'] == 1]['Age'] , ax = ax)
sns.kdeplot(df_train[df_train['Survived'] == 0]['Age'] , ax = ax)
plt.legend(['Survived == 1' , 'Survived == 0'])
plt.show()

In [ ]:
# Age distribution within classes
plt.figure(figsize = (8 , 6))
df_train['Age'][df_train['Pclass'] == 1].plot(kind = 'kde')
df_train['Age'][df_train['Pclass'] == 2].plot(kind = 'kde')
df_train['Age'][df_train['Pclass'] == 3].plot(kind = 'kde')

plt.xlabel('Age')
plt.title('Age Distribution within classes')
plt.legend(['1st Class' , '2nd Class' , '3rd Class'])

In [ ]:
cummulate_survival_ratio = []
for i in range(1 , 80):
    cummulate_survival_ratio.append(df_train[df_train['Age'] < i]['Survived'].sum() / len(df_train[df_train['Age'] < i]['Survived']))

    plt.figure(figsize = (7 , 7))
    plt.plot(cummulate_survival_ratio)
    plt.title('Survival rate change depeding on range of Age' , y = 1.02)
    plt.ylabel('Survival rate')
    plt.xlabel('Range of Age(0 ~ x)')
    plt.show()

In [ ]:
# 2.5 Pclass , Sex , Age

f , ax = plt.subplots(1 , 2 , figsize = (18 , 8))
sns.violinplot(x = 'Pclass' , y = 'Age' , hue = 'Survived' , data = df_train , scale = 'count' , split = True , ax = ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0 , 110 , 10))
sns.violinplot(x = 'Sex' , y = 'Age' , hue = 'Survived' , data = df_train , sclae = 'count' , split = True , ax = ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0 , 110 , 10))
plt.show()

In [ ]:
# 2.6 Embarked

f , ax = plt.subplots(1 , 1 , figsize = (7 , 7))
df_train[['Embarked' , 'Survived']].groupby(['Embarked'] , as_index = True).mean().sort_values(by = 'Survived' , ascending = False).plot.bar(ax = ax)

In [ ]:
f , ax = plt.subplots(2 , 2 , figsize = (20 , 15))
sns.countplot(x = 'Embarked' , data = df_train , ax = ax[0 , 0])
ax[0 , 0].set_title('(1) No. Of Passengers Boarded')
sns.countplot(x = 'Embarked' , hue = 'Sex' , data = df_train , ax = ax[0 , 1])
ax[0 , 1].set_title('(2) Male_Female Split for Embarked')
sns.countplot(x = 'Embarked' , hue = 'Survived' , data = df_train , ax = ax[1 , 0])
ax[1 , 0].set_title('(3) Embarked vs Survived')
sns.countplot(x = 'Embarked' , hue = 'Pclass' , data = df_train , ax = ax[1 , 1])
ax[1 , 1].set_title('(4) Embarked vs Pclass')
plt.subplots_adjust(wspace = 0.2 , hspace = 0.5)
plt.show()

In [ ]:
# 2.7 Family - SibSp(형제 자매) + Parch(부모 , 자녀)

df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1 # 자신을 포함해야하니 1을 더합니다
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1 # 자신을 포함해야하니 1을 더합니다

In [ ]:
print('Maximum size of Family: ' , df_train['FamilySize'].max())
print('Minimum size of Family: ' , df_train['FamilySize'].min())

In [ ]:
f , ax = plt.subplots(1 , 3 , figsize = (40 , 10))
sns.countplot(x = 'FamilySize' , data = df_train , ax = ax[0])
ax[0].set_title('(1) No. Of Passengers Boarded' , y = 1.02)

sns.countplot(x = 'FamilySize' , hue = 'Survived' , data = df_train , ax = ax[1])
ax[1].set_title('(2) Survived countplot depending on FamilySize' , y = 1.02)

df_train[['FamilySize' , 'Survived']].groupby(['FamilySize'] , as_index  = True).mean().sort_values(by = 'Survived' , ascending = False).plot.bar(ax = ax[2])
ax[2].set_title('(3) Survived rate depending on FamilySize' , y = 1.02)

plt.subplots_adjust(wspace = 0.2 , hspace = 0.5)
plt.show()

In [ ]:
# 2.8 Fare

fig , ax = plt.subplots(1 , 1 , figsize = (8 , 8))
g = sns.distplot(df_train['Fare'] , color = 'b' , label = 'Skewness : {:.2f}'.format(df_train['Fare'].skew()) , ax = ax)
g = g.legend(loc = 'best')

In [ ]:
df_test.loc[df_test.Fare.isnull() , 'Fare'] = df_test['Fare'].mean()

df_train['Fare'] = df_train['Fare'].map(lambda i : np.log(i) if i > 0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i : np.log(i) if i > 0 else 0)

In [ ]:
fig , ax = plt.subplots(1 , 1 , figsize = (8 , 8))
g = sns.histplot(
    df_train['Fare'] , color = 'b' , label = 'Skewness : {:.2f}'.format(df_train['Fare'].skew()) ,
    ax = ax , kde = True
)
g = g.legend(loc = 'best')

In [ ]:
# 2.9 Cabin

df_train.head()

In [ ]:
# 2.10 Ticket

df_train['Ticket'].value_counts()